<h1>Capstone Project</h1>

<h2>Stock Prediction Using Twitter Sentiment Analysis</h2>

### Load the datatset

In [1]:
import pandas as pd

fname = open('finalData.csv', 'r')
df = pd.read_csv(fname)
df.head()

,S.no.,Date,Open,High,Low,Close,neg,pos
0,0,2009-06-18,8496.73,8590.52,8475.12,8555.60,0.6436,0.3564
1,1,2009-06-19,8556.96,8616.59,8496.73,8539.73,0.6369,0.3631
2,2,2009-06-20,8547.74,8577.71,8415.64,8439.37,0.6488,0.3512
3,3,2009-06-21,8543.13,8558.27,8375.09,8389.19,0.6476,0.3524
4,4,2009-06-22,8538.52,8538.83,8334.55,8339.01,0.6476,0.3524


In [2]:
'''
make the train and testing dataset.
'''
train_df = df[:int(len(df)*0.8)]
test_df = df[int(len(df)*0.8):]

In [3]:
train_df.loc[1]['Low']

8496.7299999999996

In [4]:
def build_set(t_df):
    count = len(t_df)
    data_row = {'Date':[], 'Open':[], 'High': [], 'Low':[], 'Close':[], 'neg': [], 'pos': [], 'target': []}
    for i in range(count-1):
        data_row['Date'].append(t_df.loc[i]['Date'])
        data_row['Open'].append(t_df.loc[i]['Open'])
        data_row['High'].append(t_df.loc[i]['High'])
        data_row['Low'].append(t_df.loc[i]['Low'])
        data_row['Close'].append(t_df.loc[i]['Close'])
        data_row['neg'].append(t_df.loc[i]['neg'])
        data_row['pos'].append(t_df.loc[i]['pos'])
        data_row['target'].append(t_df.loc[i+1]['Close'])
    
#     return data_row
    final_df = pd.DataFrame.from_dict(data=data_row,orient='columns', dtype=None)
    return final_df

In [62]:
train_data = build_set(train_df)

In [63]:
train_targets = train_data['target']
train_data_labels = train_data.drop(['Date' ,'target'], axis=1)
train_data_labels.head()

,Close,High,Low,Open,neg,pos
0,8555.60,8590.52,8475.12,8496.73,0.6436,0.3564
1,8539.73,8616.59,8496.73,8556.96,0.6369,0.3631
2,8439.37,8577.71,8415.64,8547.74,0.6488,0.3512
3,8389.19,8558.27,8375.09,8543.13,0.6476,0.3524
4,8339.01,8538.83,8334.55,8538.52,0.6476,0.3524


### Treating it as a supervised classification problem

In [64]:
for index, row in train_data.iterrows():
    if row['target'] > row['Close']:
        train_data.ix[index, 'target'] = 1
    else:
        train_data.ix[index, 'target'] = 0

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  """


In [65]:
train_data.head()

,Close,Date,High,Low,Open,neg,pos,target
0,8555.60,2009-06-18,8590.52,8475.12,8496.73,0.6436,0.3564,0.0
1,8539.73,2009-06-19,8616.59,8496.73,8556.96,0.6369,0.3631,0.0
2,8439.37,2009-06-20,8577.71,8415.64,8547.74,0.6488,0.3512,0.0
3,8389.19,2009-06-21,8558.27,8375.09,8543.13,0.6476,0.3524,0.0
4,8339.01,2009-06-22,8538.83,8334.55,8538.52,0.6476,0.3524,0.0


In [66]:
from sklearn.linear_model import LogisticRegression


clf = LogisticRegression(random_state=3)
clf.fit(train_data_labels, train_targets)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=3, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [67]:
clf.predict(train_data_labels.iloc[0])

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([ 1.])

In [68]:
test_df.index = range(len(test_df))
# test_df.head()
test_data = build_set(test_df)
test_targets = test_data['target']
test_data_labels = test_data.drop(['Date' ,'target'], axis=1)
test_data_labels.head()

,Close,High,Low,Open,neg,pos
0,10246.97,10260.80,10197.69,10223.01,0.6494,0.3506
1,10291.26,10341.97,10246.14,10247.42,0.6335,0.3665
2,10197.47,10321.64,10171.24,10289.82,0.6495,0.3505
3,10270.47,10305.69,10192.03,10197.85,0.6869,0.3131
4,10338.72,10369.97,10229.74,10232.84,0.6549,0.3451


In [69]:
for index, row in test_data.iterrows():
    if row['target'] > row['Close']:
        test_data.ix[index, 'target'] = 1
    else:
        test_data.ix[index, 'target'] = 0

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  This is separate from the ipykernel package so we can avoid doing imports until


In [74]:
lgr_score = clf.score(test_data_labels, test_targets)
print("The Logistic Regression return an accuracy of {:.2f}%".format(lgr_score*100))

The Logistic Regression return an accuracy of 63.89%
